<a href="https://colab.research.google.com/github/Sidhtang/bert-project/blob/main/fast_api_for_the_mode_l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
from transformers import AutoTokenizer
import os

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/distilbert_general_router_model.pth"

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    print(f"Error: Model file not found or empty: {model_name}")
else:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    try:
        model = torch.load(model_name, map_location=device)
    except RuntimeError as e:
        print(f"Error loading model: {e}")
    else:
        model.eval()  # Put the model in evaluation mode
        model.to(device)

        # Function for inference
        def classify_query(query):
            inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            return "Personalization" if prediction == 1 else "Customer_support"

        # Test the model with queries
        queries = [
            "Hey there, you guys got some nice hoodies for me?"
        ]
        for query in queries:
            result = classify_query(query)
            print(f"Query: {query}")
            print(f"Prediction: {result}")
            print()

Using device: cpu


<ipython-input-9-fa02934a2366>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name, map_location=device)


Query: Hey there, you guys got some nice hoodies for me?
Prediction: Personalization



In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import asyncio
import uvicorn

# Initialize FastAPI app
app = FastAPI(title="Text Classification API")

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/distilbert_general_router_model.pth"

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    raise FileNotFoundError(f"Error: Model file not found or empty: {model_name}")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

try:
    model = torch.load(model_name, map_location=device)
    model.eval()  # Put the model in evaluation mode
    model.to(device)
except RuntimeError as e:
    raise RuntimeError(f"Error loading model: {e}")

class Query(BaseModel):
    text: str

def classify_query(query: str):
    inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "Personalization" if prediction == 1 else "Customer_support"

@app.post("/classify")
async def classify_text(query: Query):
    try:
        result = classify_query(query.text)
        return {"query": query.text, "classification": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {"message": "Welcome to the Text Classification API"}

if __name__ == "__main__":
    # Use uvicorn.Config to create a server instance
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)

    # Use asyncio to run the server in the current event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

<ipython-input-16-59a6f1b2141d>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name, map_location=device)


Using device: cpu


RuntimeError: This event loop is already running